In [ ]:
%pip install TensorFlow
%pip install Keras
%pip install pillow
%pip install NumPy
%pip install tqdm
%pip install jupyterlab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.5 MB/s eta 0:00:00
     ━━

In [ ]:
import numpy as np
import pandas as pd
import io
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from keras.layers.Concatenate(axis=-1)
from keras.models import Model, load_model
from keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm as tqdm #to check loop progress
tqdm().pandas()

0it [00:00, ?it/s]

In [ ]:
# Load the document file into memory
def load_fp(filename):
  with open(filename) as file:
    text = file.read()
  return text

In [ ]:
# Get all images with captiosn
def img_capt(filename):
  with open(filename) as file:
    captions = file.split('n')
    descriptions = {}
    for caption in captions[:-1]:
      img, caption = caption.split('t')
      if img[:-2] not in descriptions:
        descriptions[img[:-2]] = [caption]
      else:
        descriptions[img[:-2]].append(caption)
  return descriptions

In [ ]:
# Clean text from captions
def txt_cleaning(captions):
  table = str.maketrans('','',string.punctuation)
  for img, caps in captions.items():
    for i, img_caption in enumerate(caps):
              img_caption.replace("-"," ")
              descp = img_caption.split()
              #uppercase to lowercase
              descp = [wrd.lower() for wrd in descp]
              #remove punctuation from each token
              descp = [wrd.translate(table) for wrd in descp]
              #remove hanging 's and a
              descp = [wrd for wrd in descp if (len(wrd)>1)]
              #remove words containing numbers with them
              descp = [wrd for wrd in descp if (wrd.isalpha())]
              #converting back to string
              img_caption = ' '.join(descp)
              captions[img][i]= img_caption
    return captions

In [ ]:
# Build vocab of unique words
def txt_vocab(descriptions):
  vocab = set()
  for key in descriptions.keys():
    [vocab.update(d.split()) for d in descriptions[key]]
  return vocab

In [ ]:
# Save descriptions in single file
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
    for desc in desc_list:
      lines.append(key + 't' + desc )
  data = "n".join(lines)
  with open(filename) as file:
    file.write(data)

In [3]:
from google.colab import files
uploaded = files.upload()
dataset_text = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))
# Dataset is now stored in a Pandas Dataframe

Saving Flickr8k.token.txt to Flickr8k.token.txt


NameError: ignored

In [ ]:
uploaded = files.upload()
dataset_images = pd.read_csv(io.BytesIO(uploaded['Filename.csv']))

In [ ]:
#to prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#map them into descriptions dictionary 
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
#to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")